In [1]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

In [12]:
SAVE = False

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

We need this in a dataframe, which we build from the `text` string like so:

In [3]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


We will take all samples from `sentence_A` and build sentence embeddings for each - which we can then store in `faiss`.

In [4]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

And we will pull in the `sentence_B` column too, giving us ~4.5k unique sentences.

In [5]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

This isn't a particularly large number, so let's pull in a few more similar datasets.

In [6]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [7]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [8]:
len(set(sentences))

14505

Before converting to our sentence embeddings, we will save to text file as backup.

In [9]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
]

In [ ]:
if SAVE:
    with open('sentences.txt', 'w') as fp:
        fp.write('\n'.join(sentences))

Now we have 14.5k unique sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time...)

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

(14504, 768)

We can save/load from file in the case fo needing to reload the notebook for any reason later.

In [11]:
sentence_embeddings.shape[0]

14504

In [13]:
if SAVE:
    with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[0:256])

In [15]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    if SAVE:
        with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
            np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

In [16]:
d = sentence_embeddings.shape[1]
d

768

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is to specify the vector dimensionality - which in this case is `d==768` (to align with the sentence-BERT model output embeddings of size `768`).

In [17]:
index = faiss.IndexFlatL2(d)

Often, we will use indexes that require us to `train` them on our data before being used (if we are grouping or transforming the data in any way). `IndexFlatL2` however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector `xq` during search. So, in this case, no training is required - which we can confirm by checking the `is_trained` attribute.

In [18]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [19]:
index.add(sentence_embeddings)

In [20]:
index.ntotal

14504

Then search given a query `xq` and the number of nearest neighbors to return `k`.

In [21]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [22]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  

[[ 5977  7573 12191   605]]
CPU times: user 13.7 ms, sys: 58 µs, total: 13.8 ms
Wall time: 12.2 ms


In [23]:
[f'{i}: {sentences[i]}' for i in I[0]]

['5977: A group of football players is running in the field',
 '7573: A group of people playing football is running in the field',
 '12191: Two groups of people are playing football',
 '605: A person playing football is running past an official carrying a football']

In [24]:
sentences[5977]

'A group of football players is running in the field'

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from faiss, we can do that too.

In [25]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [26]:
vecs.shape

(4, 768)

In [27]:
vecs[0][:100]

array([ 0.01627038,  0.22325905, -0.15037443, -0.30747256, -0.27122423,
       -0.1059317 , -0.06460934,  0.04738167, -0.73349047, -0.37657675,
       -0.76762825,  0.16902883,  0.53107637,  0.51176661,  1.14415872,
       -0.08562914, -0.67240077, -0.96637076,  0.02545444, -0.21559833,
       -1.25656569, -0.82982141, -0.09825029, -0.2185083 ,  0.50610232,
        0.10527954,  0.50396878,  0.65242964, -1.39458752,  0.65847462,
       -0.21525331, -0.22487442,  0.81818324,  0.08464327, -0.76141691,
       -0.28928319, -0.09825825, -0.73046207,  0.07855757, -0.84354657,
       -0.59242076,  0.77471346, -1.20920551, -0.2275794 , -1.30733597,
       -0.23081481, -1.31322563,  0.01629069, -0.97285485,  0.19308175,
        0.47424546,  1.18920863, -1.96741295, -0.70061159, -0.29638708,
        0.60533714,  0.62407446, -0.70340377, -0.86754227,  0.1767319 ,
       -0.19170497, -0.02951997,  0.22623591, -0.16695465, -0.80402559,
       -0.45918933,  0.69675493, -0.24928212, -1.0147866 , -0.92

# Adding Partitioning to the Index

faiss allows us to add an additional step to optimize our search effiency using variety of different methods.
A popular approach is to partition the index into [Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram).
Using this method we would take our query vector `xq`, identify the cell it belongs to, and then use our `IndexFlatL2` to search between the query vector `xq` and all indexed vectors belonging to that cell.
We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous `IndexFlatL2` operation as a quantization step (another step in the search process), and feeding this into the new `IndexIVFFlat` operation like so:

In [28]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter `nlist`. we use `nlist` to define how many partitions we'd like our index to have.

When we built the previous, `IndexFlatL2`-only index, we noted that no training was required as no grouping/tranformation was required to build that index.
Now that we've added partitioning using `IndexIVFFlat`, this is no longer the case.
Let's take a look at the `is_trained` attribute.

In [29]:
index.is_trained

False

So, what we need to do now is `train` our index on our data, which we do before adding any data to the index (otherwise the index cannot know how to group each vector).

In [30]:
index.train(sentence_embeddings)
index.is_trained

True

Now our index is trained, we add our data just as we did before.

In [31]:
index.add(sentence_embeddings)
index.ntotal

14504

Let's search again using the same indexed sentence embeddings and the same query `xq`.

In [32]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 5977  7573 12191   605]]
CPU times: user 1.34 ms, sys: 192 µs, total: 1.53 ms
Wall time: 900 µs


We can increase the number of nearby cells to serach too with `nprobe`.

In [33]:
index.nprobe = 10

In [34]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 5977  7573 12191   605]]
CPU times: user 2.39 ms, sys: 1.96 ms, total: 4.36 ms
Wall time: 3.1 ms


Increasing the number of `nprobe` will improve the accuracy of our search, but cost time. Our earlier `IndexFlatL2`-only search was exhaustive (it compared every single vector) and so it identified the closest matches with a perfect accuracy.
The smaller our `nprobe` value, the smaller scope that we search.
We received perfect results (that matched our previous `IndexFlatL2`-only results - `7460`, `10940`, `3781`, `5747`), however, if we found that we were not getting closely matching results, we could simply bump `nprobe` up further - improving accuracy, but increasing time-taken too.

For IVF (and IMI) indexes, before attempting to use the `reconstruct` method, we need to call the `make_direct_map` method - otherwise we will return a `RunetimeError`.

In [37]:
index.make_direct_map()
index.reconstruct(7460)[:100]

array([-0.42223325,  0.25445822,  2.1197274 , -0.04604523,  0.15920906,
       -0.03425917,  0.18358617, -1.2359186 ,  0.43517134, -0.41971323,
       -1.1047288 ,  0.59653074,  0.70958394,  0.10358086, -1.2594174 ,
       -0.29242876,  0.02118083, -1.034143  , -0.2594775 , -0.95105344,
       -1.311581  , -0.46893784, -0.60118765,  0.02630937, -0.1638549 ,
        0.894012  , -0.17102613,  0.43290544, -1.4750891 ,  0.69652313,
        0.01709623,  0.07306284,  0.31114212, -0.458619  , -0.8999929 ,
       -0.19464928,  0.4906521 , -0.53843707,  0.24446803,  0.59129316,
       -0.5541887 ,  0.44797096, -0.0782951 , -0.13418755, -0.17247899,
       -0.6924553 , -0.4919634 ,  0.72926825,  0.96075565, -1.1803845 ,
       -0.10572469, -0.44152603, -0.8944549 , -0.21623953, -0.8901376 ,
        0.10714849,  0.26428026, -1.5570687 , -0.6085926 , -0.21136774,
       -1.4472393 ,  0.23004436,  0.7577629 ,  0.732899  , -1.0625232 ,
        0.20482826,  0.35931626,  0.10953128,  0.30508724, -0.38

We've now significantly reduced the search time, what can we do next?

# Quantization

Well, when storing these vectors we're storing the full (e.g. `Flat`) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find thgat storing the full vectors consumes too much space.

Fortunately, faiss comes with the ability to compress our bectors using transformations based on Product Quanitzation (PQ).
But, what is PQ?
Well, we can view it as an additional approximation step similar to our use of IVF, which allowed us to approximate by reducing the scope of our search.
PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

Some tutorials [here](https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/).

PQ achieves this approximated distance operation by compressing the vectors themselves.
This consist of a few steps:

1. We split every original vector into several subvectors
2. For each set of subvector, we perform a clustering operation, creating many centroids for each subvector set
3. In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid


In [38]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [39]:
index.is_trained

False

In [40]:
index.train(sentence_embeddings)

In [41]:
index.add(sentence_embeddings)

Let's compare it to our previous index without PQ, and an `nprobe` value of `10`.

In [42]:
index.nprobe = 10

In [43]:
%%time
D, I = index.search(xq, k)
print(I)

[[1059  605 6440  768]]
CPU times: user 960 µs, sys: 0 ns, total: 960 µs
Wall time: 698 µs


Though adding PQ reduced the search time significantly, a small difference on a dataset of this size, but when scaled ot larger dataset this can make a huge difference.

Now, we should also notice the slightly different reqults being returned.
Beforehand, with our exhaustive L2 search we were returning different results that were very close.

In [44]:
[f'{i}: {sentences[i]}' for i in I[0]]

['1059: A football player is running past an official carrying a football',
 '605: A person playing football is running past an official carrying a football',
 '6440: Different teams are playing football on a field',
 '768: position in football played by a team member']